In [1]:
import sys
sys.path.append("..")

import numpy as np
import pandas as pd
import pandas_ta as ta
import MetaTrader5 as mt5

from sesto.constants import CURRENCY_PAIRS
from sesto.indicators import SMA, BB, RSI
import sesto.metatrader.data as mtd
from sesto.plot import plot_plotly


MetaTrader 5 initialized successfully.


In [2]:
PAIRS = CURRENCY_PAIRS
TIMEFRAME = mt5.TIMEFRAME_H1
BARS = 500
SMA_PERIOD = 14
RSI_OVERSOLD = 20
RSI_OVERBOUGHT = 80

In [3]:
mtd.fill_data_pos(PAIRS, TIMEFRAME, BARS)

Fetched data for pair: USDJPY
Fetched data for pair: EURUSD
Fetched data for pair: GBPUSD
Fetched data for pair: EURGBP
Fetched data for pair: CADCHF
Fetched data for pair: EURJPY
Fetched data for pair: AUDUSD
Fetched data for pair: USDCNH
Fetched data for pair: EURCHF
Fetched data for pair: NZDUSD


In [12]:
for symbol, df in mtd.data.items():
    # Custom logic to calculate the ROC, no using libraries
    # Define the ROC period
    ROC_PERIOD = 14  # You can adjust this period as needed

    # Calculate Rate of Change (ROC)
    df['roc'] = ((df['close'] - df['close'].shift(ROC_PERIOD)) / df['close'].shift(ROC_PERIOD)) * 100

    # Calculate ROC Signal Line (Simple Moving Average of ROC)
    ROC_SIGNAL_PERIOD = 9  # You can adjust this period as needed
    df['roc_signal'] = df['roc'].rolling(window=ROC_SIGNAL_PERIOD).mean()

    # Calculate ROC Histogram
    df['roc_hist'] = df['roc'] - df['roc_signal']

    # Calculate ROC Momentum (Difference of ROC)
    df['roc_momentum'] = df['roc'].diff()

    # Handle missing values by filling them with zeros or appropriate methods
    df.fillna(0, inplace=True)

    # Optional: Calculate Additional ROC-Related Indicators

    # 1. ROC Slope (Rate of Change of ROC)
    ROC_SLOPE_PERIOD = 3  # Number of periods to calculate the slope
    df['ROC_Slope'] = (df['roc'] - df['roc'].shift(ROC_SLOPE_PERIOD)) / ROC_SLOPE_PERIOD

    # 2. ROC Percentage Change
    df['roc_pct_change'] = df['roc'].pct_change() * 100

    # 3. Smoothed ROC using Exponential Moving Average (EMA)
    EMA_PERIOD = 9
    df['roc_ema'] = df['roc'].ewm(span=EMA_PERIOD, adjust=False).mean()

    # 4. ROC Rate of Change (Second Order ROC)
    df['roc_rate_of_change'] = ((df['roc'] - df['roc'].shift(ROC_PERIOD)) / df['roc'].shift(ROC_PERIOD)) * 100

    plot_plotly(df, symbol, ['roc', 'roc_signal'])
